In [1]:
# Centralizing data processing.
import pandas as pd
import os
def prepare_sales_data(file_path):
    """

    :param file_path: Path to the uploaded sales file in the data folder.
    :return: Root Mean Squared Error for all unique products presented as an average to evaluate the model
    """
    # Load the sales data
    df = pd.read_csv(file_path)
    df = pd.melt(
        df,
        id_vars=['Product Code', 'Product Name'],
        var_name='Date',
        value_name='Sales'
    )
    # Convert date columns to periods
    df['Date'] = pd.to_datetime(df['Date'])
    return df

In [10]:
""" 
Here take the sales data and merge it with the stock data so that the 'Product Code', 'Product name' and 'Date' merge
can form a point for determining finished goods variances.
Step 1: Understand the Data

    finished_goods_inventory DataFrame: Contains the initial stock (Quantity) and the date (Date) of stock take.
    sales_data_merged DataFrame: Contains sales forecasts, quantities, and stock deltas (which indicate changes in inventory).

Step 2: Identify the Relevant Date Range

We need to identify the first date of stock take (from finished_goods_inventory) and the last date from the sales_data_merged DataFrame.
Step 3: Initialize the Inventory DataFrame

We’ll create a new DataFrame to hold the cumulative inventory over the date range.
Step 4: Iterate Over the Dates and Update the Inventory

We will update the inventory based on the Stock_delta for each day.
Step 5: Create the Final Inventory DataFrame

"""
# Process the sales data for the calculation.
sales_data = prepare_sales_data(os.path.join("data", "forecast.csv"))

# Read the finished goods inventory and parse the dates into DateTime format.
finished_goods_inventory = pd.read_csv(os.path.join("data", "finished_goods_inventory.csv"))
finished_goods_inventory['Date'] = pd.to_datetime(finished_goods_inventory['Date'])

# Merge inventory data with sales data forecast on the date of inventory counts.
# The start of the forecast should not be later than than the inventory counts date.
sales_data_merged = sales_data.merge(finished_goods_inventory,on=['Product Code', 'Product Name', 'Date'],how='left')

# Retain relevant columns and remove irrelevant columns for the MRP calculation.
# Create column showing difference between forecast and inventory.
sales_data_merged = sales_data_merged.drop(columns=['Category', 'Unit of Measure', 'Description'])
sales_data_merged['Quantity'] = sales_data_merged['Quantity'].fillna(0)
sales_data_merged['Stock_delta'] = sales_data_merged['Quantity']-sales_data_merged['Sales']

import pandas as pd

# Assuming `sales_data_merged` and `finished_goods_inventory` are already loaded

# start_date should be the date of stock take (the first date in finished_goods_inventory['Date'])
start_date = finished_goods_inventory['Date'].min()

# Get the last date in the sales_data_merged DataFrame
end_date = sales_data_merged['Date'].max()

# Create a date range from start_date to end_date
date_range = pd.date_range(start=start_date, end=end_date)

# Initialize the inventory DataFrame with Product Code and Product Name as a MultiIndex
inventory_df = pd.DataFrame(index=pd.MultiIndex.from_frame(finished_goods_inventory[['Product Code', 'Product Name']].drop_duplicates()),
                            columns=date_range)

# Set the initial stock for the start_date
for index, row in finished_goods_inventory.iterrows():
    product_code = row['Product Code']
    product_name = row['Product Name']
    inventory_df.loc[(product_code, product_name), start_date] = row['Quantity']

# Fill the rest of the dates with cumulative inventory
for i in range(1, len(date_range)):
    current_date = date_range[i]
    previous_date = date_range[i - 1]

    # First, copy the previous day's inventory to the current day
    inventory_df[current_date] = inventory_df[previous_date]

    # Apply stock delta changes from sales_data_merged for the current date
    daily_sales_data = sales_data_merged[sales_data_merged['Date'] == current_date]
    
    for index, row in daily_sales_data.iterrows():
        product_code = row['Product Code']
        product_name = row['Product Name']
        inventory_df.loc[(product_code, product_name), current_date] += row['Stock_delta']

# Fill NaN values in inventory_df with the last valid inventory value (carry forward stock levels)
inventory_df = inventory_df.ffill(axis=1)

# Ensure the correct dtype inference after forward fill
inventory_df = inventory_df.infer_objects(copy=False)

print(inventory_df)


                                2024-08-18  2024-08-19  2024-08-20  \
Product Code Product Name                                            
FG001        Finished Good A          61.0        55.0        46.0   
FG002        Finished Good B          80.0        73.0        68.0   
FG003        Finished Good C          17.0         3.0       -10.0   
FG004        Finished Good D         104.0       101.0        98.0   
FG005        Finished Good E          13.0        10.0         7.0   
FG006        Finished Good F         114.0       105.0        96.0   
FG007        Finished Good G         116.0       110.0       104.0   
FG008        Finished Good H          20.0        12.0         3.0   
FG009        Finished Good I          61.0        46.0        29.0   
FG010        Finished Good J         106.0        68.0        27.0   
FG011        Finished Good K         120.0        95.0        71.0   
FG012        Finished Good L          72.0        27.0       -17.0   
FG013        Finishe

C:\Users\User\AppData\Local\Temp\ipykernel_15604\1009258418.py:78: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  inventory_df = inventory_df.ffill(axis=1)
